## Train and save a Keras h5 model

In [1]:
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from mnist import get_mnist

Using TensorFlow backend.


In [2]:
input_shape, num_classes, x_train, y_train, y_train_cat, x_test, y_test, y_test_cat = get_mnist()

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.summary()

x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1

In [3]:
model.fit(x_train, y_train_cat,
          batch_size=128,
          epochs=1,
          verbose=1,
          validation_data=(x_test, y_test_cat))

loss, accuracy = model.evaluate(x_test, y_test_cat, verbose=0)
print('Test accuracy:', accuracy)

os.makedirs('./model', exist_ok=True)
model.save('./model/mnist.h5')
print("h5 model saved")


Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 95s 2ms/step - loss: 0.2453 - accuracy: 0.9249 - val_loss: 0.0530 - val_accuracy: 0.9822
Test accuracy: 0.982200026512146
h5 model saved


## Load h5 model and convert to pb 

In [4]:
import tensorflow as tf
from keras import backend as K
from keras.models import load_model

In [5]:
# This line must be executed before loading Keras model.
K.set_learning_phase(0)

h5_model = load_model('./model/mnist.h5')
h5_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [6]:
#from show_graph import show_graph
sess = K.get_session()
graph_def = sess.graph.as_graph_def()
#show_graph(graph_def)

input_name = h5_model.inputs[0].name
output_name = h5_model.outputs[0].name

print("TAKE NOTE OF INPUT AND OUTPUT HERE!")
print("Input:", input_name)
print("Output:", output_name)

TAKE NOTE OF INPUT AND OUTPUT HERE!
Input: conv2d_1_input_1:0
Output: dense_2_1/Softmax:0


In [7]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        
        frozen_graph = convert_variables_to_constants(session, input_graph_def, output_names, freeze_var_names)
        
        return frozen_graph

In [8]:
frozen_graph = freeze_session(K.get_session(), output_names=[out.op.name for out in h5_model.outputs])
tf.train.write_graph(frozen_graph, "./model", "mnist.pb", as_text=False)

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 78 variables.
INFO:tensorflow:Converted 78 variables to const ops.


'./model\\mnist.pb'

### (restart kernel here)

## Load pb model and run prediction

In [1]:
import os
import sys
import tensorflow as tf
from tensorflow.python.platform import gfile

In [2]:
with gfile.FastGFile("./model/mnist.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

sess = tf.InteractiveSession()
sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer and place into the current default `Graph`.
tf.import_graph_def(graph_def, name='')

Instructions for updating:
Use tf.gfile.GFile.


In [3]:
for op in sess.graph.get_operations():
    #print(op.name, op.type)
    if '_input' in op.name:
        input_name = op.name + ":0"
    elif 'Softmax' in op.name:
        output_name = op.name+ ":0"

print("input:", input_name)
print("output:", output_name)

input: conv2d_1_input_1:0
output: dense_2_1/Softmax:0


In [4]:
softmax_tensor = sess.graph.get_tensor_by_name(output_name)

In [5]:
from mnist import get_mnist

input_shape, num_classes, x_train, y_train, y_train_cat, x_test, y_test, y_test_cat = get_mnist()
print(x_test.shape[0], 'test samples')

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
x_test shape: (10000, 28, 28, 1)
10000 test samples


In [6]:
predictions = sess.run(softmax_tensor, {input_name: x_test[:20]})

In [7]:
[p.argmax() for p in predictions]

[7, 2, 1, 0, 4, 1, 4, 9, 5, 9, 0, 6, 9, 0, 1, 5, 9, 7, 3, 4]

In [8]:
print("accuracy: ", sum([p.argmax() for p in predictions] == y_test[:20]) / 20)

accuracy:  1.0
